In [98]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import config

In [99]:
trips = pd.read_csv(config.TRIPS_1216_PRC, low_memory=False)
trips.shape

(44351, 20)

In [100]:
trips.head()

,TRIPID,PERSID,TRIPNO,TRAVDOW,LINKMODE,TRIPPURP,DESTPURP1,STARTHOUR,STARTIME,ARRHOUR,ARRTIME,TRIPTIME,TRAVTIME,WAITIME,ORIGSA1,ORIGSA3,DESTSA1,DESTSA3,CUMDIST,Mode
0,Y12H0000107P01T01,Y12H0000107P01,1,Sunday,Vehicle Driver,Social,Social,10,630,10,650,20,20,0,20904122028,Whittlesea - Wallan,20904122002,Whittlesea - Wallan,2.60,Car
1,Y12H0000107P01T02,Y12H0000107P01,2,Sunday,Vehicle Driver,Social,At or Go Home,12,750,12,760,10,10,0,20904122002,Whittlesea - Wallan,20904122028,Whittlesea - Wallan,2.60,Car
2,Y12H0000107P01T03,Y12H0000107P01,3,Sunday,Walking,Recreational,Recreational,13,810,14,840,30,30,0,20904122028,Whittlesea - Wallan,20904122031,Whittlesea - Wallan,3.23,Walk
3,Y12H0000107P01T04,Y12H0000107P01,4,Sunday,Walking,Recreational,At or Go Home,14,855,14,885,30,30,0,20904122031,Whittlesea - Wallan,20904122028,Whittlesea - Wallan,3.23,Walk
4,Y12H0000107P02T01,Y12H0000107P02,1,Sunday,Vehicle Passenger,Social,Social,10,630,10,650,20,20,0,20904122028,Whittlesea - Wallan,20904122002,Whittlesea - Wallan,2.60,Car


In [101]:
trips.columns

Index(['TRIPID', 'PERSID', 'TRIPNO', 'TRAVDOW', 'LINKMODE', 'TRIPPURP',
       'DESTPURP1', 'STARTHOUR', 'STARTIME', 'ARRHOUR', 'ARRTIME', 'TRIPTIME',
       'TRAVTIME', 'WAITIME', 'ORIGSA1', 'ORIGSA3', 'DESTSA1', 'DESTSA3',
       'CUMDIST', 'Mode'],
      dtype='object')

### Get SA1 shapefile

In [102]:
shape_file = "../data/raw/1270055001_sa1_2016_aust_shape.zip"
gdf = gpd.read_file(shape_file)

In [103]:
gdf.shape

(57523, 15)

In [104]:
gdf.head()

,SA1_MAIN16,SA1_7DIG16,SA2_MAIN16,SA2_5DIG16,SA2_NAME16,SA3_CODE16,SA3_NAME16,SA4_CODE16,SA4_NAME16,GCC_CODE16,GCC_NAME16,STE_CODE16,STE_NAME16,AREASQKM16,geometry
0,10102100701,1100701,101021007,11007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,362.8727,"POLYGON ((149.71174 -35.12318, 149.71184 -35.1..."
1,10102100702,1100702,101021007,11007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,229.7459,"POLYGON ((149.73421 -35.36758, 149.74475 -35.3..."
2,10102100703,1100703,101021007,11007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,2.3910,"POLYGON ((149.77998 -35.44067, 149.77996 -35.4..."
3,10102100704,1100704,101021007,11007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,1.2816,"POLYGON ((149.79744 -35.44149, 149.79778 -35.4..."
4,10102100705,1100705,101021007,11007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,1.1978,"POLYGON ((149.79498 -35.44902, 149.79517 -35.4..."


In [105]:
gdf.STE_NAME16.value_counts()

New South Wales                 18399
Victoria                        14073
Queensland                      11563
Western Australia                5984
South Australia                  4245
Tasmania                         1464
Australian Capital Territory     1147
Northern Territory                626
Other Territories                  22
Name: STE_NAME16, dtype: int64

In [106]:
# Only using SA1 from Victoria
gdf_vic = gdf[gdf.STE_NAME16 == "Victoria"]
gdf_vic.shape

(14073, 15)

In [107]:
gdf_vic.columns

Index(['SA1_MAIN16', 'SA1_7DIG16', 'SA2_MAIN16', 'SA2_5DIG16', 'SA2_NAME16',
       'SA3_CODE16', 'SA3_NAME16', 'SA4_CODE16', 'SA4_NAME16', 'GCC_CODE16',
       'GCC_NAME16', 'STE_CODE16', 'STE_NAME16', 'AREASQKM16', 'geometry'],
      dtype='object')

In [108]:
# Remove SA1 do not have geometry
gdf_vic.dropna(inplace=True)

<ipython-input-108-11880f43da31>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf_vic.dropna(inplace=True)


In [109]:
gdf_vic.isna().sum()

SA1_MAIN16    0
SA1_7DIG16    0
SA2_MAIN16    0
SA2_5DIG16    0
SA2_NAME16    0
SA3_CODE16    0
SA3_NAME16    0
SA4_CODE16    0
SA4_NAME16    0
GCC_CODE16    0
GCC_NAME16    0
STE_CODE16    0
STE_NAME16    0
AREASQKM16    0
geometry      0
dtype: int64

In [110]:
columns = ['SA1_MAIN16', 'SA2_MAIN16', 'SA2_NAME16', 
           'SA3_CODE16', 'SA3_NAME16', 'SA4_CODE16', 'SA4_NAME16', 
           'STE_NAME16', 'AREASQKM16', 'geometry']
gdf_vic = gdf_vic[columns]
gdf_vic.shape

(14069, 10)

In [111]:
gdf_vic["centroid"] = gdf_vic.geometry.apply(lambda x: x.centroid)

In [112]:
gdf_vic.head()

,SA1_MAIN16,SA2_MAIN16,SA2_NAME16,SA3_CODE16,SA3_NAME16,SA4_CODE16,SA4_NAME16,STE_NAME16,AREASQKM16,geometry,centroid
18399,20101100101,201011001,Alfredton,20101,Ballarat,201,Ballarat,Victoria,0.1805,"POLYGON ((143.80089 -37.55384, 143.80039 -37.5...",POINT (143.80268 -37.55164)
18400,20101100102,201011001,Alfredton,20101,Ballarat,201,Ballarat,Victoria,0.0728,"POLYGON ((143.79772 -37.55073, 143.79780 -37.5...",POINT (143.79768 -37.55221)
18401,20101100105,201011001,Alfredton,20101,Ballarat,201,Ballarat,Victoria,0.6349,"POLYGON ((143.81562 -37.55871, 143.81687 -37.5...",POINT (143.81432 -37.56436)
18402,20101100106,201011001,Alfredton,20101,Ballarat,201,Ballarat,Victoria,0.4161,"POLYGON ((143.80094 -37.55384, 143.80239 -37.5...",POINT (143.80248 -37.55616)
18403,20101100107,201011001,Alfredton,20101,Ballarat,201,Ballarat,Victoria,0.2292,"POLYGON ((143.81294 -37.55644, 143.81300 -37.5...",POINT (143.81556 -37.55658)


### Merge OD SA1 centroids to trips dataset

In [113]:
trips.head()

,TRIPID,PERSID,TRIPNO,TRAVDOW,LINKMODE,TRIPPURP,DESTPURP1,STARTHOUR,STARTIME,ARRHOUR,ARRTIME,TRIPTIME,TRAVTIME,WAITIME,ORIGSA1,ORIGSA3,DESTSA1,DESTSA3,CUMDIST,Mode
0,Y12H0000107P01T01,Y12H0000107P01,1,Sunday,Vehicle Driver,Social,Social,10,630,10,650,20,20,0,20904122028,Whittlesea - Wallan,20904122002,Whittlesea - Wallan,2.60,Car
1,Y12H0000107P01T02,Y12H0000107P01,2,Sunday,Vehicle Driver,Social,At or Go Home,12,750,12,760,10,10,0,20904122002,Whittlesea - Wallan,20904122028,Whittlesea - Wallan,2.60,Car
2,Y12H0000107P01T03,Y12H0000107P01,3,Sunday,Walking,Recreational,Recreational,13,810,14,840,30,30,0,20904122028,Whittlesea - Wallan,20904122031,Whittlesea - Wallan,3.23,Walk
3,Y12H0000107P01T04,Y12H0000107P01,4,Sunday,Walking,Recreational,At or Go Home,14,855,14,885,30,30,0,20904122031,Whittlesea - Wallan,20904122028,Whittlesea - Wallan,3.23,Walk
4,Y12H0000107P02T01,Y12H0000107P02,1,Sunday,Vehicle Passenger,Social,Social,10,630,10,650,20,20,0,20904122028,Whittlesea - Wallan,20904122002,Whittlesea - Wallan,2.60,Car


In [114]:
trips = trips.merge(gdf_vic[['SA1_MAIN16','centroid']], how="left", left_on="ORIGSA1", right_on="SA1_MAIN16")
trips = trips.rename(columns={"centroid":"ORIG_CENTROID"})
trips = trips.drop(columns=["SA1_MAIN16"], axis=1)

In [115]:
trips = trips.merge(gdf_vic[['SA1_MAIN16','centroid']], how="left", left_on="DESTSA1", right_on="SA1_MAIN16")
trips = trips.rename(columns={"centroid":"DEST_CENTROID"})
trips = trips.drop(columns=["SA1_MAIN16"], axis=1)

In [116]:
trips.head()

,TRIPID,PERSID,TRIPNO,TRAVDOW,LINKMODE,TRIPPURP,DESTPURP1,STARTHOUR,STARTIME,ARRHOUR,...,TRAVTIME,WAITIME,ORIGSA1,ORIGSA3,DESTSA1,DESTSA3,CUMDIST,Mode,ORIG_CENTROID,DEST_CENTROID
0,Y12H0000107P01T01,Y12H0000107P01,1,Sunday,Vehicle Driver,Social,Social,10,630,10,...,20,0,20904122028,Whittlesea - Wallan,20904122002,Whittlesea - Wallan,2.60,Car,POINT (145.08730 -37.65974),POINT (145.07152 -37.65148)
1,Y12H0000107P01T02,Y12H0000107P01,2,Sunday,Vehicle Driver,Social,At or Go Home,12,750,12,...,10,0,20904122002,Whittlesea - Wallan,20904122028,Whittlesea - Wallan,2.60,Car,POINT (145.07152 -37.65148),POINT (145.08730 -37.65974)
2,Y12H0000107P01T03,Y12H0000107P01,3,Sunday,Walking,Recreational,Recreational,13,810,14,...,30,0,20904122028,Whittlesea - Wallan,20904122031,Whittlesea - Wallan,3.23,Walk,POINT (145.08730 -37.65974),POINT (145.06223 -37.66497)
3,Y12H0000107P01T04,Y12H0000107P01,4,Sunday,Walking,Recreational,At or Go Home,14,855,14,...,30,0,20904122031,Whittlesea - Wallan,20904122028,Whittlesea - Wallan,3.23,Walk,POINT (145.06223 -37.66497),POINT (145.08730 -37.65974)
4,Y12H0000107P02T01,Y12H0000107P02,1,Sunday,Vehicle Passenger,Social,Social,10,630,10,...,20,0,20904122028,Whittlesea - Wallan,20904122002,Whittlesea - Wallan,2.60,Car,POINT (145.08730 -37.65974),POINT (145.07152 -37.65148)


In [117]:
trips.TRAVDOW.value_counts()

Thursday     7119
Wednesday    7024
Friday       6920
Monday       6798
Tuesday      6529
Saturday     5233
Sunday       4728
Name: TRAVDOW, dtype: int64

### Saves trips OD locations into CSV

In [118]:
trips.to_csv("../data/processed/trip_od_locations.csv", index=False)

In [119]:
trips.shape

(44351, 22)

### Get person and household main info

##### Get household info

In [120]:
hh = pd.read_csv("../data/raw/VISTA_2012_16_v1_SA1_CSV/H_VISTA12_16_SA1_V1.csv")
hh.shape

/home/danph/.pyenv/versions/3.8.5/envs/.ml/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (32) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(18152, 53)

In [121]:
cols = ['HHID', 'HHSIZE', 'VISITORS', 'DWELLTYPE',
       'OWNDWELL', 'YEARSLIVED', 'MONTHSLIVED', 'ADULTBIKES', 'KIDSBIKES',
       'TOTALBIKES', 'PHONE', 'CARS', 'FOURWDS', 'UTES', 'VANS', 'TRUCKS',
       'MBIKES', 'OTHERVEHS', 'TOTALVEHS', 'TRAVDATE', 'TRAVMONTH', 'TRAVYEAR',
       'HOMETAZ', 'YOUNGEST', 'AVEAGE', 'OLDEST', 'HHINC']

In [122]:
hh = hh[cols]
hh.shape

(18152, 27)

##### Get person info

In [123]:
ps = pd.read_csv("../data/raw/VISTA_2012_16_v1_SA1_CSV/P_VISTA12_16_SA1_V1.csv")
ps.shape

(46562, 78)

In [124]:
cols =['PERSID', 'HHID', 'PERSNO', 'WEEK', 'MONTHOFBIRTH',
       'YEAROFBIRTH', 'AGE', 'AGEGROUP', 'SEX', 'RELATIONSHIP', 'CARLICENCE',
       'MBIKELICENCE', 'OTHERLICENCE', 'NOLICENCE', 'FULLTIMEWORK',
       'PARTTIMEWORK', 'CASUALWORK', 'ANYWORK', 'STUDYING', 'ACTIVITIES',
       'MAINACT', 'WORKTYPE', 'EMPTYPE', 'ANZSCO1', 'ANZSIC1',
       'STARTPLACE', 'ANYSTOPS', 'NUMSTOPS', 'REASONCODE', 'MRTDOW',
       'MRTINT', 'PERSINC', 'WHOFILLED', 'PROXY', 'FILLDOW', 'FILLLAG',
       'ADDITIONALTRAVEL', 'NEWSTOP', 'DISABLEDTRANS', 'TAXITRANS',
       'WHEELCHAIRTRANS', 'ACCESSTRANS', 'NONETRANS', 'NOTTOSAYTRANS',
       'CYCLEDWORK', 'CYCLEDSHOPPING', 'CYCLEDEXERCISE', 'CYCLEDOTHER',
       'NOCYCLED', 'HOMESLA', 'HOMELGA', 'HomeSA1', 'HomeSA2', 'HomeSA3',
       'HomeSA4', 'HomeRegion_ASGS', 'HomeSubRegion_ASGS', 'AgeGroup_RW', 'STOPS',
       'ReportingPeriod']

In [125]:
ps = ps[cols]
ps.shape

(46562, 60)

##### Merge household info into person table

In [126]:
ps = ps.merge(hh, how='left', on='HHID')
ps.shape

(46562, 86)

In [127]:
ps.to_csv("../data/processed/persons.csv", index=False)

### Merge person info into Trip table

In [131]:
trips = trips.merge(ps, how="left", on='PERSID')
trips.shape

(44351, 107)

In [132]:
trips.to_csv("../data/processed/trip_od_full.csv", index=False)